# **Dependecies**

In [11]:
import torch
from torch import nn
from transformers import BertModel, BertTokenizer

# **Common Setup Functions**


In [12]:
def setup_model_and_tokenizer(model_name='bert-base-uncased'):
    model = BertModel.from_pretrained(model_name)
    tokenizer = BertTokenizer.from_pretrained(model_name)
    return model, tokenizer

def prepare_inputs(text, tokenizer):
    return tokenizer(text, return_tensors='pt')

def create_additional_input_vector(hidden_size):
    return torch.randn(1, 1, hidden_size)

# **Common Integration Method Apllier**

In [13]:
class IntegrationMethodApplier(nn.Module):
    def __init__(self, integration_method):
        super(IntegrationMethodApplier, self).__init__()
        self.integration_method = integration_method

    def forward(self, input_tensor, additional_input_vector):
        if self.integration_method == "addition":
            return input_tensor + additional_input_vector
        elif self.integration_method == "multiplication":
            return input_tensor * additional_input_vector
        else:
            raise ValueError("Unsupported integration method")

# **Hook-Based Approach**

## **Define the HookBasedBERTModifier**

In [14]:
class HookBasedBERTModifier:
    def __init__(self, model, layer_number, integration_method_applier):
        self.model = model
        self.layer_number = layer_number
        self.integration_method_applier = integration_method_applier
        self.hook = None

    def modify_input(self, module, input):
        input_tensor = input[0]
        modified_input = self.integration_method_applier(input_tensor, self.additional_input_vector)
        return (modified_input,)

    def register_hook(self, additional_input_vector):
        self.additional_input_vector = additional_input_vector
        layer = self.model.encoder.layer[self.layer_number - 1]
        self.hook = layer.register_forward_pre_hook(self.modify_input)

    def remove_hook(self):
        if self.hook is not None:
            self.hook.remove()
            self.hook = None

## **Test Hook-Based Approach**

In [17]:
def test_hook_based_modifier(input_text, layer_number, integration_method):
    model, tokenizer = setup_model_and_tokenizer()
    additional_input_vector = create_additional_input_vector(model.config.hidden_size)
    integration_method_applier = IntegrationMethodApplier(integration_method)

    modifier = HookBasedBERTModifier(model, layer_number, integration_method_applier)

    inputs = prepare_inputs(input_text, tokenizer)

    with torch.no_grad():
        outputs_without_hook = model(**inputs)

    modifier.register_hook(additional_input_vector)

    with torch.no_grad():
        outputs_with_hook = model(**inputs)

    modifier.remove_hook()

    output_difference = torch.abs(outputs_with_hook.last_hidden_state - outputs_without_hook.last_hidden_state)
    print("Output difference: ", torch.sum(output_difference).item())
    # return outputs_without_hook, outputs_with_hook

## **Run The Test**

In [18]:
test_hook_based_modifier("Hello, how are you?", 10, "multiplication")

Output difference:  3657.418212890625


# **Custom Layer-Based Approach**


## **Define the CustomLayerBERTModifier**

In [19]:
class CustomLayerBERTModifier(nn.Module):
    def __init__(self, model, layer_number, integration_method_applier):

        super(CustomLayerBERTModifier, self).__init__()
        self.bert = model
        self.layer_number = layer_number
        self.integration_method_applier = integration_method_applier

    def forward(self, input_ids, attention_mask, additional_input_vector):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
        hidden_states = outputs.hidden_states
        modified_layer_input = self.integration_method_applier(hidden_states[self.layer_number - 1], additional_input_vector)
        for i in range(self.layer_number, len(self.bert.encoder.layer)):
            modified_layer_input = self.bert.encoder.layer[i](modified_layer_input)[0]

        return modified_layer_input


## **Test Custom Layer-Based Approach**

In [23]:
def test_custom_layer_modifier(input_text, layer_number, integration_method):
    model, tokenizer = setup_model_and_tokenizer()

    additional_input_vector = create_additional_input_vector(model.config.hidden_size)

    integration_method_applier = IntegrationMethodApplier(integration_method)
    custom_model = CustomLayerBERTModifier(model, layer_number, integration_method_applier)

    inputs = prepare_inputs(input_text, tokenizer)

    with torch.no_grad():
        outputs = custom_model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], additional_input_vector=additional_input_vector)

    print("Output shape: ", outputs.shape)
    # return outputs

## **Run The Test**

In [24]:
test_custom_layer_modifier("Hello, how are you?", 10, "addition")

Output shape:  torch.Size([1, 8, 768])
